In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb 28

In [ ]:
@author: Kuldeep Kumar
"""

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from matplotlib import pyplot
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
df_train = pd.read_excel(r'train_agriculture.xlsx')
df_test = pd.read_excel(r'test_agriculture.xlsx')

In [ ]:
print("Column headings:")
print(df_train.columns)

In [ ]:
print("Look into Data",df_train.head(10))

In [ ]:
print("\n Shape of the train dataset",df_train.shape)

Missing value in dataset

In [ ]:
print("\n Missing Value in train data set" ,df_train.isnull().sum())
print("\n Missing Value in test data set" ,df_test.isnull().sum())

Class distribution

In [ ]:
print("\n Class Distribution counts ",df_train['Crop_Damage'].value_counts())
print("\n Class Distribution counts ",df_train.groupby('Crop_Damage').size())

Concatenation of Data train and test after that split of data will do and Seprate the X and Y<br>
rint(df_train.shape)<br>
rint(df_test.shape)

In [ ]:
df_train['train_test']='training_data'
df_test['train_test']='testing_data'

In [ ]:
df_data = pd.concat((df_train, df_test),sort=False)

In [ ]:
print("\n Look into data after concatenate", df_data.head(10))

Null value check in dataset after concatenate

In [ ]:
print("Null Valve in dataset after concatenate", df_data.isnull().sum())

In [ ]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(df_data['Number_Weeks_Used'].values.reshape(-1,1))
imp_mean.fit(df_data['Crop_Damage'].values.reshape(-1,1))

In [ ]:
df_data['Number_Weeks_Used'] = imp_mean.transform(df_data['Number_Weeks_Used'].values.reshape(-1,1))
df_data['Crop_Damage'] = imp_mean.transform(df_data['Crop_Damage'].values.reshape(-1,1))

In [ ]:
print("Null Valve in dataset after Impute", df_data.isnull().sum())

Look into data after impute nan with mean values

In [ ]:
print("Look into data after impute nan with mean values ",df_data.head(10))
print("Colums after impute", df_data.columns)

Drop Id field having no value added for data

In [ ]:
df_data.drop('ID',axis=1,inplace=True)
print("Data and shape after drop of id",df_data.shape)

Split-out validation dataset

In [ ]:
train_data=df_data.loc[df_data.train_test.isin(['training_data'])]
test_data=df_data.loc[df_data.train_test.isin(['testing_data'])]

In [ ]:
train_data.drop(['train_test'],axis=1,inplace=True)
test_data.drop(['train_test'],axis=1,inplace=True)

In [ ]:
print("train and test shape",train_data.shape, test_data.shape)

visualize the correlations between the attributes.<br>
correlation matrix

In [ ]:
plt.figure(figsize=(14,12))
sns.heatmap(train_data.corr(),linewidths=.1,cmap="Purples", annot=True, annot_kws={"size": 7})
plt.yticks(rotation=0)
plt.savefig("corr.png", format='png', dpi=900, bbox_inches='tight')

In [ ]:
X_train, Y_train = train_data.drop(["Crop_Damage"], axis=1).values, train_data["Crop_Damage"].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=44)

In [ ]:
print("Shape of test and train data", X_train.shape, Y_train.shape, X_test.shape)

In [ ]:
models = []
models.append(("LR",LogisticRegression()))
models.append(("NB",GaussianNB()))
models.append(("RF",RandomForestClassifier()))
models.append(("SVC",SVC()))
models.append(("Dtree",DecisionTreeClassifier()))
models.append(("XGB",xgb.XGBClassifier()))
models.append(("KNN",KNeighborsClassifier()))

In [ ]:
results = []
names = []
for name,model in models:
    kfold = KFold(n_splits=2, random_state=22,shuffle=True)
    cv_result = cross_val_score(model,X_train,y_train, cv = kfold,scoring = "accuracy")
    #print(name, cv_result)
    results.append(cv_result)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_result.mean(), cv_result.std())
    print(msg)

Output for Accuracy

R: 0.836912 (0.000044)<br>
B: 0.817613 (0.002496)<br>
F: 0.825225 (0.003486)<br>
VC: 0.832563 (0.000046)<br>
tree: 0.742595 (0.005638)<br>
GB: 0.815439 (0.001409)<br>
NN: 0.818972 (0.003311)<br>
ccuracy: 0.837 (0.000)

Compare Algorithms

In [ ]:
fig = pyplot.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

######### Logistic Regression Model #########

In [ ]:
Lkfold = KFold(n_splits=2, random_state=22,shuffle=True)
Lmodel = LogisticRegression()
#################Accuracy Check for Logistic as per spot check of Algo##########
scoring = 'accuracy'
results = cross_val_score(Lmodel, X_train,y_train, cv=Lkfold, scoring=scoring)
print("Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))
#########prediction on test data#######
ns_probs = [0 for _ in range(len(y_test))]
Lmodel.fit(X_train, y_train)
lr_probs = Lmodel.predict_proba(X_test)
preds=np.argmax(lr_probs,axis=1)
##########prediction on full data and contenation with ID########
a=test_data.drop(['Crop_Damage'],axis=1)
lr_probs_test = Lmodel.predict_proba(a)
preds_test=np.argmax(lr_probs_test,axis=1)
test_data['Crop_Damage']=preds_test
combine_test= pd.concat([df_test['ID'], test_data['Crop_Damage']], axis=1)
combine_test.to_csv("FinalSubmitOutput.csv",index=False)

######################<br>
Standardize the dataset

In [ ]:
num_folds=2
seed=22

In [ ]:
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR', LogisticRegression())])))
pipelines.append(('ScaledNB', Pipeline([('Scaler', StandardScaler()),('NB', GaussianNB())])))
pipelines.append(('ScaledRF', Pipeline([('Scaler', StandardScaler()),('EN',RandomForestClassifier())])))
pipelines.append(('ScaledSVC', Pipeline([('Scaler', StandardScaler()),('SVC', SVC())])))
pipelines.append(('ScaledDtree', Pipeline([('Scaler', StandardScaler()),('Dtree',DecisionTreeClassifier())])))
pipelines.append(('ScaledXGB', Pipeline([('Scaler', StandardScaler()),('XGB',xgb.XGBClassifier())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsClassifier())])))

In [ ]:
results = []
names = []

In [ ]:
for name, model in pipelines:
    kfold = KFold(n_splits=num_folds, random_state=seed,shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
# Output is
#ScaledLR: 0.828214 (0.000047)
#ScaledNB: 0.817613 (0.002496)
#ScaledRF: 0.823321 (0.001679)
#ScaledSVC: 0.839359 (0.001859)
#ScaledDtree: 0.743410 (0.004823)
#ScaledXGB: 0.815439 (0.001409)
#ScaledKNN: 0.828487 (0.003487)
## Compare Algorithms

In [ ]:
fig = pyplot.figure()
fig.suptitle('Scaled Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

fter scaled of data SVC is having more accuracy that 83% now do the Tunning of SVC<br>
and its observed from output that it is lightly inproved 83 to 84 going with Ensemble<br>
technique before finalizing the Model<br>
Tune scaled SVM

In [ ]:
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
c_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0]
kernel_values = ['linear', 'poly', 'rbf', 'sigmoid']
param_grid = dict(C=c_values, kernel=kernel_values)
model = SVC()
kfold = KFold(n_splits=num_folds, random_state=seed,shuffle=True)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

utput is<br>
est: 0.840175 using {'C': 2.0, 'kernel': 'rbf'}<br>
.832563 (0.000046) with: {'C': 0.1, 'kernel': 'linear'}<br>
.839903 (0.002403) with: {'C': 0.1, 'kernel': 'poly'}<br>
.832563 (0.000046) with: {'C': 0.1, 'kernel': 'rbf'}<br>
.831476 (0.001041) with: {'C': 0.1, 'kernel': 'sigmoid'}<br>
.832563 (0.000046) with: {'C': 0.3, 'kernel': 'linear'}<br>
.839903 (0.002403) with: {'C': 0.3, 'kernel': 'poly'}<br>
.836368 (0.001675) with: {'C': 0.3, 'kernel': 'rbf'}<br>
.797230 (0.007556) with: {'C': 0.3, 'kernel': 'sigmoid'}<br>
.832563 (0.000046) with: {'C': 0.5, 'kernel': 'linear'}<br>
.839903 (0.002403) with: {'C': 0.5, 'kernel': 'poly'}<br>
.838000 (0.000500) with: {'C': 0.5, 'kernel': 'rbf'}<br>
.784726 (0.007552) with: {'C': 0.5, 'kernel': 'sigmoid'}<br>
.832563 (0.000046) with: {'C': 0.7, 'kernel': 'linear'}<br>
.839903 (0.002403) with: {'C': 0.7, 'kernel': 'poly'}<br>
.838815 (0.001315) with: {'C': 0.7, 'kernel': 'rbf'}<br>
.776029 (0.012443) with: {'C': 0.7, 'kernel': 'sigmoid'}<br>
.832563 (0.000046) with: {'C': 0.9, 'kernel': 'linear'}<br>
.839359 (0.002403) with: {'C': 0.9, 'kernel': 'poly'}<br>
.839087 (0.001587) with: {'C': 0.9, 'kernel': 'rbf'}<br>
.769778 (0.013800) with: {'C': 0.9, 'kernel': 'sigmoid'}<br>
.832563 (0.000046) with: {'C': 1.0, 'kernel': 'linear'}<br>
.839359 (0.002403) with: {'C': 1.0, 'kernel': 'poly'}<br>
.839359 (0.001859) with: {'C': 1.0, 'kernel': 'rbf'}<br>
.768691 (0.013800) with: {'C': 1.0, 'kernel': 'sigmoid'}<br>
.832563 (0.000046) with: {'C': 1.3, 'kernel': 'linear'}<br>
.839359 (0.002403) with: {'C': 1.3, 'kernel': 'poly'}<br>
.838815 (0.001859) with: {'C': 1.3, 'kernel': 'rbf'}<br>
.764342 (0.013798) with: {'C': 1.3, 'kernel': 'sigmoid'}<br>
.832563 (0.000046) with: {'C': 1.5, 'kernel': 'linear'}<br>
.839087 (0.002131) with: {'C': 1.5, 'kernel': 'poly'}<br>
.839359 (0.001859) with: {'C': 1.5, 'kernel': 'rbf'}<br>
.757003 (0.013525) with: {'C': 1.5, 'kernel': 'sigmoid'}<br>
.832563 (0.000046) with: {'C': 1.7, 'kernel': 'linear'}<br>
.839087 (0.002131) with: {'C': 1.7, 'kernel': 'poly'}<br>
.839631 (0.002131) with: {'C': 1.7, 'kernel': 'rbf'}<br>
.758090 (0.014612) with: {'C': 1.7, 'kernel': 'sigmoid'}<br>
.832563 (0.000046) with: {'C': 2.0, 'kernel': 'linear'}<br>
.838815 (0.001859) with: {'C': 2.0, 'kernel': 'poly'}<br>
.840175 (0.002675) with: {'C': 2.0, 'kernel': 'rbf'}<br>
.754557 (0.013796) with: {'C': 2.0, 'kernel': 'sigmoid'}

nsemble Methods : improve the performance of algorithms on this problem<br>
our dierent ensemble machine learning<br>
lgorithms, two boosting and two bagging methods:<br>
nsemble Methods<br>
1. Boosting Methods: AdaBoost (AB) and Gradient Boosting (GBM).<br>
2. Bagging Methods: Random Forests (RF) and Extra Trees (ET).

ensembles

In [ ]:
ensembles = []
ensembles.append(('AB', AdaBoostClassifier()))
ensembles.append(('GBM', GradientBoostingClassifier()))
ensembles.append(('RF', RandomForestClassifier()))
ensembles.append(('ET', ExtraTreesClassifier()))
results = []
names = []
for name, model in ensembles:
    kfold = KFold(n_splits=num_folds, random_state=seed,shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

utput is GBM is getting highest value that 83 % which is equal to our<br>
previos model that was creted using Logistic Regression so we can try this also to finalize the model

AB: 0.832020 (0.000046)<br>
BM: 0.836369 (0.003217)<br>
F: 0.822507 (0.001854)<br>
T: 0.815168 (0.004299)

In [ ]:
num_trees = 100
kfold = KFold(n_splits=10, random_state=seed,shuffle=True)
gbmmodel = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed)
results = cross_val_score(model,  X_train, y_train, cv=kfold)
print("Mean and Standard Deviation for GBM",results.mean(),results.std())

#######prediction on test data#######

In [ ]:
ns_probs = [0 for _ in range(len(y_test))]
gbmmodel.fit(X_train, y_train)
lr_probs = gbmmodel.predict_proba(X_test)
preds=np.argmax(lr_probs,axis=1)
##########prediction on full data and contenation with ID########
a=test_data.drop(['Crop_Damage'],axis=1)
lr_probs_test = gbmmodel.predict_proba(a)
preds_test=np.argmax(lr_probs_test,axis=1)
test_data['Crop_Damage']=preds_test
combine_test= pd.concat([df_test['ID'], test_data['Crop_Damage']], axis=1)
combine_test.to_csv("FinalGBMSubmitOutput.csv",index=False)